# (28) Marija

**Motivation**: notebook to generate results. <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# Code path + imports
code_path = '/home/hadi/Desktop/_Ca-fMRI-Jul28'
sys.path.insert(0, code_path)
from analysis.bootstrap import *
from utils.render import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tqdm.notebook import tqdm
%matplotlib inline
set_style()

In [2]:
def get_config(**kwargs):
    _cfg, cfg = Config(**kwargs), Config(**kwargs)
    _replacement = '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis'
    dirs = [
        k for k in dir(cfg)
        if '_dir' in k and
        isinstance(getattr(cfg, k), str)
    ]
    for item in dirs:
        new = getattr(_cfg, item).replace(
            _cfg.base_dir, _replacement)
        setattr(cfg, item, new)
    cfg.svinet_dir = '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis/svinet'
    cfg.download_dir, cfg.raw_dir, cfg.tx_dir, cfg.warped_dir = None, None, None, None
    cfg._get_all_dirs()
    return cfg

In [3]:
cfg = get_config(nn=512, makedirs=False)
cfg.all_dirs

{
    'base_dir': '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis',
    'bold_dir': '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis/processed/norm-global_parcel-columnar_n-512*3/bold',
    'ca_dir': '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis/processed/norm-global_parcel-columnar_n-512*3/ca2',
    'log_dir': '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis/processed/norm-global_parcel-columnar_n-512*3/log',
    'main_dir': '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis/processed/norm-global_parcel-columnar_n-512*3',
    'masks_dir': '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis/processed/norm-global_parcel-columnar_n-512*3/masks',
    'parcel_dir': '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis/processed/norm-global_parcel-columnar_n-512*3/parcellation',
    'processed_dir': '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis/processed',
    'results_dir': '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis/processed/norm-global_parcel-columnar_n-512*3/results',
    'stim_dir': '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis/stim',
    'svinet_dir': '/ca2data3/lake_lab_analyses/MarijaHadiAnalysis/svinet'
}

In [19]:
mice.cfg.main_dir

'/ca2data3/lake_lab_analyses/MarijaHadiAnalysis/processed/norm-global_parcel-columnar_n-512*3'

## Fit network

In [4]:
task = 'rest'
bands = [None, (0.01, 0.5)]

In [5]:
mice = Mice(cfg)
props = {
    'mice': mice,
    'mode': 'ca2',
    'metric': 'pearson',
    'percentiles': np.linspace(25, 10, 4).astype(int),
    'prep_data': True,
    'binarize': False,
    'verbose': False,
}
kws = {
    'task': task,
    'desc_ca2': 'preproc',
    'runs_only': True,
    'exclude': True,
}

ROI lookup not found, time to run organize_func() . . .

combine metadata not found, time to run cat_data() . . .

In [ ]:
pbar1 = tqdm(
    bands,
    leave=True,
    total=len(bands),
)
for b in pbar1:
    kws['band_ca2'] = b
    mice.setup_func_data(**kws)
    pbar2 = tqdm(
        mice.get_data_containers('ca2')[0],
        leave=False,
    )
    for key in pbar2:
        msg = f"running net (task-{props['task']}, "
        msg += f"b-{str(b).replace(' ', '')})"
        msg += f": {key}"
        pbar1.set_description(msg)
        net = Network(key=key, **props)
        net.fit_network(force=True, full=False, save=True)

## Apply SVINET (bash)

Description here

## Fit SVINET

In [ ]:
num_k = 7
perc = 'p15-sample'
bands = [None, (0.01, 0.5)]
task = 'rest'

In [ ]:
mice = Mice(cfg)
props = {
    'mice': mice,
    'num_k': num_k,
    'perc': perc,
    'mode': 'ca2',
    'metric': 'pearson',
    'match_metric': 'euclidean',
    'match_using': 'gam',
    'graph_type': 'real',
    'verbose': False,
}
kws = {
    'task': task,
    'desc_ca2': 'preproc',
    'runs_only': True,
    'exclude': True,
}

In [ ]:
pbar1 = tqdm(
    bands,
    leave=True,
    total=len(bands),
)
for b in pbar1:
    kws['band_ca2'] = b
    mice.setup_func_data(**kws)
    proc, *_ = mice.get_data_containers('ca2')
    pbar2 = tqdm(
        proc.items(),
        leave=False,
    )
    for key in pbar2:
        msg = f"running net (task-{task}, "
        msg += f"b-{str(b).replace(' ', '')})"
        msg += f": {key}"
        pbar1.set_description(msg)
        _ = SVINET(key=key, **props).fit_svinet()

## Fit Group results

In [ ]:
num_k = 7
perc = 'p15-sample'
bands = [None, (0.01, 0.5)]
task = 'rest'

In [ ]:
mice = Mice(cfg)
props = {
    'mice': mice,
    'num_k': num_k,
    'perc': perc,
    'mode': 'ca2',
    'metric': 'pearson',
    'sv_props': {
        'match_metric': 'euclidean',
        'match_using': 'gam'},
    'graph_type': 'real',
    'dist_metric': 'cosine',
    'ordering': 'infer',
    'verbose': False,
}
kws = {
    'task': task,
    'desc_ca2': 'preproc',
    'runs_only': True,
    'exclude': True,
}

In [ ]:
pbar = tqdm(
    bands,
    position=0,
    leave=True,
    dynamic_ncols=True,
    total=len(bands),
)
for b in pbar:
    msg = f"running group (task-{task}, "
    msg += f"b-{str(b).replace(' ', '')}, kk-{kk})"
    pbar.set_description(msg)
    
    mice.setup_func_data(band_ca2=b)
    gr = Group(**props)
    if gr.svinets_exist():
        gr.fit_group(force=False)

## Plot results